In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

In [46]:
folder_path = 'tests_data'
output_folder = 'plots'
os.makedirs(output_folder, exist_ok=True) # create if doesnt exist
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

Different algorithms comparison

In [ ]:
for file in excel_files:
    # read and tidy file
    df = pd.read_excel(file, header=None)
    df = df.iloc[:, [0, 3]]
    df.columns = ['benchmark', 'time']
    df[['benchmark', 'size']] = df['benchmark'].str.split('/', n=1, expand=True)
    df['size'] = pd.to_numeric(df['size'], errors='coerce')
    df['time'] = df['time'] * 10e-9 # convert to seconds

    # get plot size name
    filename = os.path.basename(file)
    name_only = os.path.splitext(filename)[0]
    if len(name_only) >= 5:
        rest = name_only[4:]
        if '_' in rest:
            underscore_index = rest.index('_')
            plot_label = rest[:underscore_index]
            plot_label2 = rest[underscore_index + 1:]
        else:
            plot_label = rest
            plot_label2 = '?'
    else:
        plot_label = '?'
        plot_label2 = '?'

    if plot_label2 == 'power':
        plot_label3 = 'potęg'
    elif plot_label2 == 'mult':
        plot_label3 = 'wielokrotności'
    else:
        plot_label3 = plot_label2

    # draw plot - each unique benchmark value as one line
    for bench in df['benchmark'].unique():
        # if bench == 'BM_Strassen3_200':
        #     continue
        bench_df = df[df['benchmark'] == bench].copy()
        bench_df = bench_df.sort_values('size')
        plt.plot(bench_df['size'], bench_df['time'], marker='o', label=bench)
    
    plt.xlabel('Wymiar macierzy')
    plt.ylabel('Czas (sekundy)')
    if plot_label != '500':
        plt.yscale('log')
    # plt.xscale('log', base=float(plot_label))
    plt.title(f'Macierze wymiaru kolejnych {plot_label3} {plot_label}')
    plt.legend(title='Benchmark')
    plt.grid(True)
    plt.tight_layout()

    # save plot
    output_path = os.path.join(output_folder, f'plot_{plot_label}_{plot_label2}.png')
    plt.savefig(output_path)
    plt.close()

Strassen3 & naive algorithm complexity

In [ ]:
for file in ['test100_mult.xlsx', 'test3_power.xlsx']:
    df = pd.read_excel(os.path.join(folder_path, file), header=None)
    df = df.iloc[:, [0, 3]]
    df.columns = ['benchmark', 'time']
    df[['benchmark', 'size']] = df['benchmark'].str.split('/', n=1, expand=True)
    df['size'] = pd.to_numeric(df['size'], errors='coerce')
    df['time'] = df['time'] * 10e-9 # convert to seconds

     # get plot size name
    filename = os.path.basename(file)
    name_only = os.path.splitext(filename)[0]
    if len(name_only) >= 5:
        rest = name_only[4:]
        if '_' in rest:
            underscore_index = rest.index('_')
            plot_label = rest[:underscore_index]
            plot_label2 = rest[underscore_index + 1:]
        else:
            plot_label = rest
            plot_label2 = '?'
    else:
        plot_label = '?'
        plot_label2 = '?'

    strassen_df = df[df['benchmark'] == 'BM_Strassen3']
    strassen_df = strassen_df.sort_values('size')

    plt.plot(strassen_df['size'], strassen_df['time'], marker='o', label='strassen3')
    plt.xlabel('Wymiar macierzy')
    plt.ylabel('Czas (sekundy)')
    plt.title(f'Złożoność obliczeniowa algorytmu strassen3')
    plt.grid(True)
    plt.tight_layout()
    # plt.yscale('log')
    # add log_3(23) plot
    a = np.log(23) / np.log(3)
    x_vals = np.linspace(df['size'].min(), df['size'].max(), 200)
    y_vals = (x_vals ** a) * 1.7e-6
    plt.plot(x_vals, y_vals, '--', color='black', label=r'$O(n^{\log_3 23}$)')
    plt.legend()
    # save plot
    output_path = os.path.join(output_folder, f'plot_strassen3_{plot_label}_{plot_label2}_complexity.png')
    plt.savefig(output_path)
    plt.close()

    naive_df = df[df['benchmark'] == 'BM_Trivial']
    naive_df = naive_df.sort_values('size')

    plt.plot(naive_df['size'], naive_df['time'], marker='o', label='algorytm naiwny')
    plt.xlabel('Wymiar macierzy')
    plt.ylabel('Czas (sekundy)')
    plt.title(f'Złożoność obliczeniowa algorytmu naiwnego')
    plt.grid(True)
    plt.tight_layout()
    # plt.yscale('log')
    # add log_3(23) plot
    x_vals = np.linspace(df['size'].min(), df['size'].max(), 200)
    y_vals = (x_vals ** 3) * 3e-8
    plt.plot(x_vals, y_vals, '--', color='black', label=r'$O(n^{3}$)')
    plt.legend()
    # save plot
    output_path = os.path.join(output_folder, f'plot_trivial_{plot_label}_{plot_label2}_complexity.png')
    plt.savefig(output_path)
    plt.close()